In [ ]:
import pandas as pd
import networkx as nx
from modules import ps

In [3]:
# Suppress warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [11]:
G = nx.read_edgelist("Denmark_edgelist.csv", delimiter=",")

num_components = nx.number_connected_components(G)

# Get the connected components (as a list of sets of nodes)
connected_components = list(nx.connected_components(G))

# Print the nodes in each connected component
for i, component in enumerate(connected_components, 1):
    print(f"Component {i}: {component}")

Component 1: {'127', '15774', '11702', '20747', '57', '18696', '172', '20559', '273', '20388', '18707', '21083', '189', '18720', '20349', '18712', '20496', '145', '141', '74', '130', '214', '19000', '20370', '121', '82', '17629', '20364', '15763', '20373', '18713', '49', '93', '152', '208', '18278', '20820', '213', '173', '252', '235', '97', '15788', '260', '20781', '15770', '20941', '20767', '20360', '134', '20361', '20916', '201', '18722', '20363', '48', '20390', '20358', '154', '19928', '21075', '257', '224', '109', '18725', '219', '18723', '20398', '244', '15776', '20350', '20635', '197', '17628', '217', '15787', '118', '21044', '20798', '99', '238', '220', '20376', '286', '120', '20381', '20372', '20389', '18724', '20415', '20841', '20527', '16728', '18694', '20356', '180', '16503', '17141', '207', '1146', '20966', '110', '20962', '199', '19920', '20355', '20779', '20362', '278', '164', '1845', '20629', '18709', '16582', '18719', '162', '20367', '18693', '12', '178', '168', '80', 

In [12]:
# Function to assign opinion scores based on voting records
def compute_opinion_scores(vote_matrix_path):
    df = pd.read_csv(vote_matrix_path, index_col=0)  # Load the voting matrix
    opinions = df.mean(axis=1).to_dict()  # Compute average vote per node
    return opinions

# Function to add weights to edges based on node opinions
def add_weights(edge_list_path, opinions):
    G = nx.read_edgelist(edge_list_path, delimiter=",")  # Load graph

    # Set edge weights as the average opinion of connected nodes
    edge_weights = {e: (opinions[e[0]] + opinions[e[1]]) / 2 for e in G.edges if e[0] in opinions and e[1] in opinions}
    nx.set_edge_attributes(G, edge_weights, name="pol")

    # Save updated edgelist with weights
    weighted_edge_list_path = edge_list_path.replace(".csv", "_wpol.csv")
    nx.write_edgelist(G, weighted_edge_list_path, delimiter=",", data=["pol"])
    print(f"Weighted edgelist saved to {weighted_edge_list_path}")

# Function to compute polarization score
def calc_pol(edge_list_path, opinions):
    G = nx.read_edgelist(edge_list_path, delimiter=",")  # Load graph
    o = {node: opinions[node] for node in G.nodes if node in opinions}  # Filter nodes with opinions
    return ps.ge(o, {}, G)  # Compute polarization score

# Main execution
if __name__ == "__main__":
    edge_list_file = "Denmark_edgelist.csv"  # Replace with your actual file
    vote_matrix_file = "sparse_matrix_denmark.csv"  # Replace with your actual file

    opinions = compute_opinion_scores(vote_matrix_file)
    add_weights(edge_list_file, opinions)
    pol_score = calc_pol(edge_list_file, opinions)

    print(f"Polarization score: {pol_score}")

Weighted edgelist saved to Denmark_edgelist_wpol.csv
Polarization score: 0.0
